In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets
from torchvision import transforms

https://www.bilibili.com/video/BV18g4119737/?p=75&spm_id_from=pageDriver&vd_source=70200f7d09862fd682e5f89b22c89125

# 1. CIFAR-10概述
1. CIFAR-10（sai far ten）很小，教学很好用，10类*6k张照片
2. 只有32*32的大小


# 2. LeNet-5
1. ![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/202401202014124.png)
## 3.1 几个注意点
1. 分类问题使用CrossEntropyLoss，Regression用MSE
2. `forward`函数中：
    - `x = self.conv_unit(x)`实际上是`x = self.conv_unit.forward(x)`，只不过`nn.Module`帮我们做了这种事
    - `logits`一般作为进入softmax之前的数值。即未经过归一化的概率
        - 先有logits，而后通过sigmoid函数或者softmax函数得到概率 p 。

In [9]:
class Lenet5(nn.Module):
    '''

    '''

    def __init__(self) -> None:
        super(Lenet5,self).__init__()

        self.conv_unit = nn.Sequential(
            # x : [32, 3, 32, 32]

            nn.Conv2d(3,6,kernel_size=5,stride=1,padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(6,16,kernel_size = 5,stride=1,padding=0),
            nn.AvgPool2d(kernel_size=2,stride=2,padding=0)
        )

        self.fc_unit = nn.Sequential(
            nn.Linear(16*5*5, 120),     # 16*5*5是后来填写的
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.ReLU()
        )

        # 分类问题使用CrossEntropyLoss
        # Regression用MSE
        self.criteon = nn.CrossEntropyLoss()

    def forward(self, x):
        '''

        :param x:  [B, 3, 32, 32]
        :return: [B, 10]
        '''

        # [B, 3, 32, 32] --> [B, 16, 5, 5]
        x = self.conv_unit(x)
        # [B, 16, 5, 5] --> [B, 16*5*5]
        x = torch.flatten(x,start_dim=1)
        # [B, 16*5*5] --> [B, 10]
        # logits: 进入Softmax之前记为logits
        logits = self.fc_unit(x)

        pred = F.softmax(logits,dim=1)

        # return logits
        return pred
        # loss = self.criteon(pred)


In [10]:
net = Lenet5()
tmp = torch.randn((32,3,32,32))
out = net(tmp)

# 3. Main
1. 技巧：我们希望data是normalized的形状
    - 本质上是希望training 和 testing都具有很好且一致的分布
    - 但是实际上属于作弊：提前看到了testing data中的数据
    -

In [11]:
batchsz = 32
EPOCH = 2
device = torch.device('cuda')

In [16]:
def main():

    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
                    transforms.Resize((32,32)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
                ]), download=True)
    cifar_train = DataLoader(cifar_train,batch_size = batchsz, shuffle = True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
                    transforms.Resize((32,32)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])
                ]), download=True)
    cifar_test = DataLoader(cifar_test,batch_size = batchsz, shuffle = True)

    # # 读取第1个iter生成的迭代对象
    # x, label = next(iter(cifar_train))
    # print(x.shape)

    model = Lenet5().to(device)
    criteon = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    for epoch in range(EPOCH):
        for batchidx, (x,lable) in enumerate(cifar_train):
            x,lable = x.to(device), lable.to(device)
            logits = model(x)
            # logits 是还没有经过
            loss = criteon(logits, lable)

            optimizer.zero_grad()   # Init
            loss.backward()         # Calculate
            optimizer.step()        # Backprop

        print(f"epoch={epoch}, loss = {loss.detach().item()}")

    model.eval()
    with torch.no_grad():
        ######### Test
        total_correct = 0
        for x, label in cifar_test:
            x,lable = x.to(device), lable.to(device)

            # [B,10]
            logits = model(x)
            # [B]
            pred = logits.argmax(dim=1)
            total_correct = (pred == label).sum()
            # total_correct = torch.eq(pred, label).float().sum()
            print(f"total_correct = {total_correct}")


In [17]:
main()

Files already downloaded and verified
Files already downloaded and verified
epoch=0, loss = 2.142469644546509
epoch=1, loss = 2.1181437969207764
epoch=2, loss = 2.069254159927368
epoch=3, loss = 2.264920234680176
epoch=4, loss = 2.11747670173645


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

DRAFT DRAFT DRAFT DRAFT DRAFT

In [25]:
weight = 32
padding = 0
kernel = 5
stride = 1
(weight+2*padding-kernel)/stride + 1

28.0